In [1]:
# 匯入套件
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from bs4 import BeautifulSoup
import requests
from requests_html import HTML
import pandas as pd
import re
import csv
from pandas.core.frame import DataFrame
import folium

# 星巴克台灣店面資料爬蟲

In [2]:
final = pd.DataFrame()
options = webdriver.ChromeOptions()
options.add_argument("headless")

browser = webdriver.Chrome(executable_path='chromedriver',options=options)
browser.get("https://www.starbucks.com.tw/stores/storesearch.jspx")
hsty1=browser.find_elements(By.CSS_SELECTOR, "#selCity > option")
# print(len(hsty1))
for i in hsty1[1:23]:
    i.click()
    sleep(1)

    inputOK0 = browser.find_element(By.XPATH,'//*[@id="sbForm:doFindByRegion"]')
    inputOK0.click()
    sleep(1)

    soup = BeautifulSoup(browser.page_source, 'lxml')
#     print(soup)

    # 擷取店名資料
    dataList1=[]
    for i in range(len(soup.findAll('span',{'id':re.compile('storeName_*')}))):
        X1=soup.findAll('span', {'id': re.compile('storeName_*')})[i].string
    #     print(X1)
    #     d={'storeName':X}
        dataList1.append(X1)
    # df2 = pd.DataFrame(dataList2)

    # 擷取店面地址資料
    dataList2=[]
    for i in range(len(soup.findAll('span',{'id':re.compile('storeAdd_*')}))):
        X2=soup.findAll('span', {'id': re.compile('storeAdd_*')})[i].string
    #     print(X2)
    #     d={'address':X}
        dataList2.append(X2)
    # df1 = pd.DataFrame(dataList2)


    # 擷取營業時間資料
    dataList3=[]
    for i in range(len(soup.findAll('p',{'class':re.compile('hours')}))):
        X3=soup.findAll('p',{'class':re.compile('hours')})[i].string
    #     print(X3)
    #     d={'hours':X}
        dataList3.append(X3)
    # df3 = pd.DataFrame(dataList3)

    # 擷取電話資料
    dataList4=[]
    for i in range(len(soup.findAll('a',{'class':re.compile('tel')}))):
        X4=soup.findAll('a',{'class':re.compile('tel')})[i].string
    #     print(X4)
    #     d={'tel':X}
        dataList4.append(X4)
    # df4 = pd.DataFrame(dataList4)

    # 擷取GPS資料
    dataList5=[]
    for i in range(len(soup.findAll('span',{'id':re.compile('storeGps_*')}))):
        X5=soup.findAll('span', {'id': re.compile('storeGps_*')})[i].string
    #     print(X5)
    #     d={'address':X5}
        dataList5.append(X5)
    # df1 = pd.DataFrame(dataList5)

    # 將上面擷取的五份資料做成一個DataFrame
    c={"storeName" : dataList1,
       "storeAdd" : dataList2,
       "hours" : dataList3,
       "tel" : dataList4,
       "storeGps": dataList5
       }

    data=DataFrame(c)
    # print(data)

    # 分割GPS欄位
    data['rel-store-lat']=data['storeGps'].str.split(',',True).str.get(0)
    data['rel-store-lng']=data['storeGps'].str.split(',',True).str.get(1)
    final2=data.drop(['storeGps'], axis=1)
    # print(final2)
    
    # 將每個縣市店面DataFrame一個一個合併起來
    final=final.append(final2,ignore_index=True) 
    # final2.to_csv('starbucklist.csv', mode='a+',encoding='utf-8_sig',header=False)

browser.quit()

final

,storeName,storeAdd,hours,tel,rel-store-lat,rel-store-lng
0,基隆南站門市,基隆市仁愛區港西街5號,(一~日)07:00~21:00,02-24259351,25.131427,121.73849
1,基隆麥金門市,基隆市安樂區麥金路201號B1,(一~日)07:00~22:00,02-24346726,25.121608,121.721250
2,基隆新豐門市,基隆市中正區新豐街162巷309號,(一~五) 07:30~21:00 (六~日) 07:30~21:00,02-24698552,25.1382825,121.7848951
3,義14門市,基隆市中正區義一路14號,(一~日)07:00~22:00,02-2427-8583,25.131181,121.744601
4,海景門市,基隆市仁愛區港西街6之2號,(一~日)07:00~22:00,02-2428-4223,25.131944,121.740438
...,...,...,...,...,...,...
494,清境門市,南投縣仁愛鄉定遠新村26-1號1樓,(一~五)08:30~21:00 (六~日)07:30~21:00,049-280-3798,24.043907,121.156931
495,澎湖喜來登門市,澎湖縣馬公市新店路197號1樓,(一~日)07:30~21:30,06-9268509,23.567903,119.576060
496,金門伯玉門市,金門縣金寧鄉伯玉路2段223號,(一~日)08:30~21:30,082-371115,24.440283,118.337664
497,金門金湖門市,金門縣金湖鎮太湖路二段198號,(日~四)09:00~18:00 (五~六)09:00~20:00,082-333457,24.439572,118.418112


In [3]:
final.to_csv('starbucklist.csv', mode='a+',encoding='utf-8_sig',header=True)

In [4]:
pd.set_option('display.max_rows', None)
final

,storeName,storeAdd,hours,tel,rel-store-lat,rel-store-lng
0,基隆南站門市,基隆市仁愛區港西街5號,(一~日)07:00~21:00,02-24259351,25.131427,121.73849
1,基隆麥金門市,基隆市安樂區麥金路201號B1,(一~日)07:00~22:00,02-24346726,25.121608,121.721250
2,基隆新豐門市,基隆市中正區新豐街162巷309號,(一~五) 07:30~21:00 (六~日) 07:30~21:00,02-24698552,25.1382825,121.7848951
3,義14門市,基隆市中正區義一路14號,(一~日)07:00~22:00,02-2427-8583,25.131181,121.744601
4,海景門市,基隆市仁愛區港西街6之2號,(一~日)07:00~22:00,02-2428-4223,25.131944,121.740438
5,101 35F門市,台北市信義區信義路5段7號35樓之1 (台北101大樓內35樓),(一~五)07:30~18:30 (六~日)及連續假期和國定節日不營業 (請提前一天致電門市...,02-8101-0701,25.033718,121.56481
6,101典藏門市,台北市信義區市府路45號B1,(一~四)07:30~21:30，週五07:30~22:00，週六 08:00~22:00，...,02-8101-8511,25.03293,121.564492
7,Bellavita-B1門市,台北市信義區松仁路28號B1,(日~四)10:30~22:00 (五~六)10:30~22:30,02-2723-3390,25.039297,121.567501
8,SOGO復興B1門市,台北市大安區忠孝東路三段300號B1,(日~四) 11:00~21:30 (五~六) 11:00~22:00,02-87727909,25.041501,121.543089
9,三創門市,台北市中正區市民大道三段2號,週日~週四 09:00~21:30 週五~週六 09:00~22:00,02-2341-8205,25.044731,121.531253


# 星巴克台灣地圖繪製

In [5]:
# 中文店名有亂碼問題，在這裡進行轉碼
def parse_zhch(s):
    return str(str(s).encode('ascii' , 'xmlcharrefreplace'))[2:-1]

# 定義繪製地圖中心位置和縮放大小
m = folium.Map((23.973875, 120.982024),zoom_start=7)

# 用迴圈把前面做好的店面DataFrame資料帶進來
i = 0
while i <=495:
    i = i + 1
    mrt = (final.iloc[i,4],final.iloc[i,5])
    name = final.iloc[i,0]
    folium.Circle(
        location=mrt,
        radius=100,
        popup=parse_zhch(name),
        color='green',
        fill=False,
        fill_color='green',
        fill_opacity=0.05,
    ).add_to(m)
m